In [5]:
from pathlib import Path
import shutil
import papermill as pm

In [ ]:
import psutil

# Get memory information
memoryInfo = psutil.virtual_memory()
total_memory = memoryInfo.total / (1024 ** 3)
available_memory = memoryInfo.available / (1024 ** 3)
used_memory = memoryInfo.used / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")
print(f"Used Memory: {used_memory:.2f} GB")

# Run `generate_nrrd_volumes` function

In [21]:
def generate_preview_nrrd_volumes(sample_dir, tiff_subdir='TIFF Stacks'):
    sample_path = Path(sample_dir)
    tiff_path = sample_path / tiff_subdir
    output_dir = sample_path / 'microCT_volume_preview'

    paper_dict = dict(image_directory_name=str(tiff_path),
                      output_dir_path=str(output_dir))

    # Run preview quality notebook
    input_nb = '00_tif2nrrd_previewQuality.ipynb'
    output_nb = sample_path / input_nb
    print(input_nb)
    pm.execute_notebook(input_nb, str(output_nb), parameters=paper_dict)

    # Copy `01_set_segment_mask.ipynb` script
    # Define source and destination paths
    source_path = '01_set_segment_mask.ipynb'
    destination_path = sample_path / source_path 
    
    # Copy the file
    shutil.copy(source_path, destination_path)

In [22]:
def generate_nrrd_volumes(sample_dir, tiff_subdir='TIFF Stacks', delete_tiff=False):
    sample_path = Path(sample_dir)
    tiff_path = sample_path / tiff_subdir

    output_dir = sample_path / 'microCT_volume'

    paper_dict = dict(image_directory_name=str(tiff_path),
                  output_dir_path=str(output_dir))
    # Run full quality notebook
    input_nb = '00_tif2nrrd_fullQuality.ipynb'
    output_nb = sample_path / input_nb
    print(input_nb)
    pm.execute_notebook(input_nb, str(output_nb), parameters=paper_dict)

    # Optionally delete TIFF directory
    if delete_tiff:
        try:
            shutil.rmtree(tiff_path)
            print(f"Deleted directory: {tiff_path}")
        except Exception as e:
            print(f"Error deleting directory {tiff_path}: {e}")


In [ ]:
sample_dir = '/config/Downloads/Gomez_4p5/'
tiff_subdir = 'TIFF Stacks'

# generate_preview_nrrd_volumes(sample_dir, tiff_subdir)
generate_nrrd_volumes(sample_dir, tiff_subdir, delete_tiff=True)

# Run `microCT_analyzer` function

In [11]:
def microCT_analyzer(sample_dir):    

    sample_path = Path(sample_dir)

    # 02_inspect_thresholds
    paper_dict = dict(volume_file = str(sample_path / 'microCT_volume/microCT_volume.nrrd'),
                      output_dir_path = str(sample_path / 'segmented_volumes/'),
                 )

    input_nb = '02_inspect_thresholds.ipynb'
    output_nb =  sample_path / input_nb

    print(input_nb)
    pm.execute_notebook(
       input_nb,
       str(output_nb),
       parameters=dict(paper_dict)
    );
    
    # 03_segment_microCT
    paper_dict = dict(volume_file = str(sample_path / 'microCT_volume/microCT_volume.nrrd'),
                      output_dir_path = str(sample_path / 'segmented_volumes/'),
                      segmentMask_file = str(sample_path / 'segmented_volumes/Segment_mask.seg.nrrd'),
                      segments_greyvalues_file = str(sample_path / 'segmented_volumes/segments_greyvalues.csv')
                 )

    input_nb = '03_segment_microCT.ipynb'
    output_nb =  str(sample_path / input_nb)

    print(input_nb)
    pm.execute_notebook(
       input_nb,
       output_nb,
       parameters=dict(paper_dict)
    );
    
    # 04_segment_polygonDefects
    paper_dict = dict(volume_file = str(sample_path / 'microCT_volume/microCT_volume.nrrd'),
                      file_nuclei_segmentation = str(sample_path / 'segmented_volumes/Bone.seg.nrrd'),
                      output_polygon_dir_path =  str(sample_path / 'segmented_volumes/polygonDefect/'),
                      sample_name = sample_dir.split('/')[-2],
                      camera_view_file =  str(sample_path / 'segmented_volumes/camera_view.csv')
                     )
    
    input_nb = '04_segment_polygonDefects.ipynb'
    output_nb =  str(sample_path / input_nb)

    print(input_nb)
    pm.execute_notebook(
       input_nb,
       output_nb,
       parameters=dict(paper_dict)
    );

    # 05_calvaria_3Dvisualizer    
    paper_dict = dict(model_dir = sample_dir
                     )
    
    input_nb = '05_calvaria_3Dvisualizer.ipynb'
    output_nb =  str(sample_path / input_nb)

    print(input_nb)
    pm.execute_notebook(
       input_nb,
       output_nb,
       parameters=dict(paper_dict)
    );

In [ ]:
sample_dir = '/config/Downloads/Gomez 5p4/'

microCT_analyzer(sample_dir)